### Load required libraries and datasets


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime
from statistics import stdev
from scipy.stats import t


In [3]:
#### Point the filePath to where you have downloaded the datasets to and 
data = pd.read_csv('QVI_data1.csv')

In [4]:
data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream
...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget


In [5]:
data.isnull().values.any()

False

# Select control stores

The client has selected stores 77, 86 and 88 as trial stores and want control stores to be established stores that are operational for the entire observation period.

We would want to match trial stores to control stores that are similar to the trial store prior to the trial period of Feb 2019 in terms of:

- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

Let's first create the metrics of interest and filter out stores that are present throughout the pre-trial period.

In [6]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [7]:
#Adding month column by splittind date column
data['Month']= data['DATE'].dt.month


In [8]:
#Monthly overall sales revenue
store_df = pd.DataFrame(data.groupby(['STORE_NBR','Month']).TOT_SALES.sum())


In [9]:
#Monthly number of customers
store_df['Monthly_no_of_cust']= pd.DataFrame(data.groupby(['STORE_NBR','Month']).LYLTY_CARD_NBR.nunique())


In [10]:
#finding same Transaction Id rows
store_dup = data[data.duplicated(['TXN_ID'])]

In [11]:
store_dup

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,Month
501,1446,2019-06-20,1,517,9,Kettle Tortilla ChpsBtroot&Ricotta 150g,2,9.2,150,KETTLE,OLDER FAMILIES,Premium,6
612,2034,2018-12-20,2,628,104,Infuzions Thai SweetChili PotatoMix 110g,1,3.8,110,INFUZIONS,YOUNG SINGLES/COUPLES,Budget,12
1114,3008,2018-08-26,3,1142,33,Cobs Popd Swt/Chlli &Sr/Cream Chips 110g,2,7.6,110,COBS,RETIREES,Budget,8
1134,3014,2019-01-09,3,1162,4,Dorito Corn Chp Supreme 380g,2,13.0,380,DORITOS,YOUNG SINGLES/COUPLES,Mainstream,1
1135,3014,2019-01-09,3,1162,15,Twisties Cheese 270g,2,9.2,270,TWISTIES,YOUNG SINGLES/COUPLES,Mainstream,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263972,271194,2019-02-23,271,269372,3,Kettle Sensations Camembert & Fig 150g,2,9.2,150,KETTLE,RETIREES,Budget,2
264064,271211,2018-12-05,271,269466,78,Thins Chips Salt & Vinegar 175g,2,6.6,175,THINS,OLDER FAMILIES,Premium,12
264112,271219,2018-08-14,271,269517,3,Kettle Sensations Camembert & Fig 150g,1,4.6,150,KETTLE,MIDAGE SINGLES/COUPLES,Mainstream,8
264453,272166,2018-10-03,272,269865,16,Smiths Crinkle Chips Salt & Vinegar 330g,2,11.4,330,SMITHS,OLDER SINGLES/COUPLES,Premium,10


In [12]:
data.drop_duplicates(subset ="TXN_ID", 
                     keep = False, inplace = True)

In [13]:
data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,Month
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,9
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,3
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,3
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream,12
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium,10
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget,10
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget,10


In [14]:
#Monthly number of transactions per customer
store_df['no_of_transaction_per_person'] = pd.DataFrame(data.groupby(['STORE_NBR','Month']).TXN_ID.nunique()/store_df['Monthly_no_of_cust'])


In [15]:
store_df

TOT_SALES  Monthly_no_of_cust  no_of_transaction_per_person
STORE_NBR Month                                                             
1         1         154.80                  35                      1.028571
          2         225.40                  52                      1.057692
          3         192.90                  45                      1.088889
          4         192.90                  42                      1.023810
          5         221.40                  46                      1.108696
...                    ...                 ...                           ...
272       8         372.85                  44                      1.090909
          9         304.70                  32                      1.125000
          10        430.60                  44                      1.113636
          11        376.20                  41                      1.097561
          12        403.90                  47                      1.000000

[3169 rows x 3 columns]

In [16]:
#No_of_sales_per_Transaction
store_df['no_of_prod_sold_per_transaction'] = pd.DataFrame(data.groupby(['STORE_NBR','Month']).PROD_QTY.sum()/data.groupby(['STORE_NBR','Month']).TXN_ID.nunique())


In [17]:
#Average_price-per_unit
store_df['avg_price_per_unit'] = pd.DataFrame(data.groupby(['STORE_NBR','Month']).TOT_SALES.sum()/data.groupby(['STORE_NBR','Month']).PROD_QTY.sum())

In [18]:
store_df

TOT_SALES  Monthly_no_of_cust  no_of_transaction_per_person  \
STORE_NBR Month                                                                
1         1         154.80                  35                      1.028571   
          2         225.40                  52                      1.057692   
          3         192.90                  45                      1.088889   
          4         192.90                  42                      1.023810   
          5         221.40                  46                      1.108696   
...                    ...                 ...                           ...   
272       8         372.85                  44                      1.090909   
          9         304.70                  32                      1.125000   
          10        430.60                  44                      1.113636   
          11        376.20                  41                      1.097561   
          12        403.90                  47                      1.000000   

                 no_of_prod_sold_per_transaction  avg_price_per_unit  
STORE_NBR Month                                                       
1         1                             1.166667            3.685714  
          2                             1.181818            3.467692  
          3                             1.183673            3.325862  
          4                             1.325581            3.384211  
          5                             1.294118            3.354545  
...                                          ...                 ...  
272       8                             1.770833            4.386471  
          9                             1.972222            4.291549  
          10                            1.938776            4.334737  
          11                            1.933333            4.324138  
          12                            1.893617            4.538202  

[3169 rows x 5 columns]

In [19]:
store_df.isnull().sum()

TOT_SALES                          0
Monthly_no_of_cust                 0
no_of_transaction_per_person       0
no_of_prod_sold_per_transaction    0
avg_price_per_unit                 0
dtype: int64

In [20]:
store_df.reset_index(inplace=True)

In [21]:
store_df

,STORE_NBR,Month,TOT_SALES,Monthly_no_of_cust,no_of_transaction_per_person,no_of_prod_sold_per_transaction,avg_price_per_unit
0,1,1,154.80,35,1.028571,1.166667,3.685714
1,1,2,225.40,52,1.057692,1.181818,3.467692
2,1,3,192.90,45,1.088889,1.183673,3.325862
3,1,4,192.90,42,1.023810,1.325581,3.384211
4,1,5,221.40,46,1.108696,1.294118,3.354545
...,...,...,...,...,...,...,...
3164,272,8,372.85,44,1.090909,1.770833,4.386471
3165,272,9,304.70,32,1.125000,1.972222,4.291549
3166,272,10,430.60,44,1.113636,1.938776,4.334737
3167,272,11,376.20,41,1.097561,1.933333,4.324138


In [22]:
#creating Dataframe of pretrial measure
Pretrialmeasure_1 = store_df.loc[store_df['Month']>6, :]

In [23]:
Pretrialmeasure_1

,STORE_NBR,Month,TOT_SALES,Monthly_no_of_cust,no_of_transaction_per_person,no_of_prod_sold_per_transaction,avg_price_per_unit
6,1,7,206.90,49,1.061224,1.192308,3.337097
7,1,8,176.10,42,1.023810,1.255814,3.261111
8,1,9,278.80,59,1.050847,1.209677,3.717333
9,1,10,188.10,44,1.022727,1.288889,3.243103
10,1,11,192.60,46,1.021739,1.212766,3.378947
...,...,...,...,...,...,...,...
3164,272,8,372.85,44,1.090909,1.770833,4.386471
3165,272,9,304.70,32,1.125000,1.972222,4.291549
3166,272,10,430.60,44,1.113636,1.938776,4.334737
3167,272,11,376.20,41,1.097561,1.933333,4.324138


In [24]:
Pretrialmeasure_2 = store_df.loc[store_df['Month']<3, :]

In [25]:
Pretrialmeasure_2

,STORE_NBR,Month,TOT_SALES,Monthly_no_of_cust,no_of_transaction_per_person,no_of_prod_sold_per_transaction,avg_price_per_unit
0,1,1,154.8,35,1.028571,1.166667,3.685714
1,1,2,225.4,52,1.057692,1.181818,3.467692
12,2,1,162.8,43,1.046512,1.088889,3.322449
13,2,2,139.4,29,1.103448,1.156250,3.767568
24,3,1,1051.7,102,1.156863,1.949153,4.443913
...,...,...,...,...,...,...,...
3134,270,2,868.4,94,1.329787,2.000000,3.473600
3145,271,1,842.4,92,1.260870,2.000000,3.538793
3146,271,2,746.2,86,1.162791,2.000000,3.667000
3157,272,1,423.0,46,1.086957,1.920000,4.406250


In [26]:
Pretrialmeasure = pd.concat([Pretrialmeasure_2,Pretrialmeasure_1])

In [27]:
# Create a function which calculates the correlation between trial store and other stores based on a single metric

def calculateCorrelation(inputTable, metric, trial_store):
    output = pd.DataFrame({'Store1': [], 'Store2': [], 'Correlation': []})
    a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metric]
    a.reset_index(drop = True, inplace = True)
    storeNumbers = inputTable['STORE_NBR'].unique()
    for i in storeNumbers:
        b = inputTable.loc[inputTable['STORE_NBR'] == i, metric]
        b.reset_index(drop = True, inplace = True)
        output = output.append({'Store1': trial_store, 'Store2': i, 'Correlation': b.corr(a)}, ignore_index = True)
    return output

In [28]:
# Create another function which calculates a standardised magnitude difference 

def calculateMagnitudeDistance(inputTable, metric, trial_store):
    output = pd.DataFrame({'Store1': [], 'Store2': [], 'Magnitude' : []})
    a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metric]
    a.reset_index(drop = True, inplace = True)
    storeNumbers = inputTable['STORE_NBR'].unique()
    for i in storeNumbers:
        b = inputTable.loc[inputTable['STORE_NBR'] == i, metric]
        b.reset_index(drop = True, inplace = True)
        c = abs(a-b)
        d = np.mean(1-(c-min(c))/(max(c)-min(c)))
        output = output.append({'Store1': trial_store, 'Store2': i, 'Magnitude': d}, ignore_index = True)
    return output

In [29]:
#how similar monthly total sales in dollar amounts for trail_store 77
trial_store = 77
corr_tot_sales = calculateCorrelation(Pretrialmeasure,'TOT_SALES',trial_store)
mag_tot_sales = calculateMagnitudeDistance(Pretrialmeasure,'TOT_SALES',trial_store)

#how similar monthly number of customers in dollar amounts for trail_store 77
corr_no_of_cust = calculateCorrelation(Pretrialmeasure,'Monthly_no_of_cust',trial_store)
mag_no_of_cust = calculateMagnitudeDistance(Pretrialmeasure,'Monthly_no_of_cust',trial_store)


c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
c:\users\burug\appdata\local\programs\python\python39\lib\site-packages\num

In [30]:
corr_tot_sales.head()

,Store1,Store2,Correlation
0,77.0,1.0,0.050163
1,77.0,2.0,-0.200796
2,77.0,3.0,0.616574
3,77.0,4.0,-0.094125
4,77.0,5.0,-0.045765


In [31]:
mag_tot_sales.head()

,Store1,Store2,Magnitude
0,77.0,1.0,0.451899
1,77.0,2.0,0.573994
2,77.0,3.0,0.569844
3,77.0,4.0,0.462734
4,77.0,5.0,0.575124


In [32]:
corr_no_of_cust.head()

,Store1,Store2,Correlation
0,77.0,1.0,0.350572
1,77.0,2.0,-0.455226
2,77.0,3.0,0.756913
3,77.0,4.0,-0.302046
4,77.0,5.0,0.277619


In [33]:
mag_no_of_cust.head()

,Store1,Store2,Magnitude
0,77.0,1.0,0.654412
1,77.0,2.0,0.544643
2,77.0,3.0,0.590909
3,77.0,4.0,0.462264
4,77.0,5.0,0.573529


In [34]:
#concatinate scores of sales together
score_sales = pd.concat([corr_tot_sales,mag_tot_sales['Magnitude']],axis=1)
score_cust = pd.concat([corr_no_of_cust,mag_no_of_cust['Magnitude']],axis=1)

In [35]:
score_sales.head()

,Store1,Store2,Correlation,Magnitude
0,77.0,1.0,0.050163,0.451899
1,77.0,2.0,-0.200796,0.573994
2,77.0,3.0,0.616574,0.569844
3,77.0,4.0,-0.094125,0.462734
4,77.0,5.0,-0.045765,0.575124


In [36]:
score_cust.head()

,Store1,Store2,Correlation,Magnitude
0,77.0,1.0,0.350572,0.654412
1,77.0,2.0,-0.455226,0.544643
2,77.0,3.0,0.756913,0.590909
3,77.0,4.0,-0.302046,0.462264
4,77.0,5.0,0.277619,0.573529


In [37]:
score_sales['scoreNsales']= 0.5*score_sales['Correlation']+0.5*score_sales['Magnitude']

In [38]:
score_cust['scoreNcust']= 0.5*score_cust['Correlation']+0.5*score_cust['Magnitude']

In [39]:
score_cust.head()

,Store1,Store2,Correlation,Magnitude,scoreNcust
0,77.0,1.0,0.350572,0.654412,0.502492
1,77.0,2.0,-0.455226,0.544643,0.044708
2,77.0,3.0,0.756913,0.590909,0.673911
3,77.0,4.0,-0.302046,0.462264,0.080109
4,77.0,5.0,0.277619,0.573529,0.425574


In [40]:
score_sales.head()

,Store1,Store2,Correlation,Magnitude,scoreNsales
0,77.0,1.0,0.050163,0.451899,0.251031
1,77.0,2.0,-0.200796,0.573994,0.186599
2,77.0,3.0,0.616574,0.569844,0.593209
3,77.0,4.0,-0.094125,0.462734,0.184304
4,77.0,5.0,-0.045765,0.575124,0.264679


In [52]:
score_control = pd.concat([score_cust['scoreNcust'],score_sales['scoreNsales']],axis=1)

In [55]:
score_control.set_index([score_sales['Store1'],score_sales['Store2']],inplace=True)

In [57]:
score_control['finalScore'] = 0.5*(score_control['scoreNcust'])+0.5*(score_control['scoreNsales'])

In [60]:
score_control.sort_values('finalScore',ascending=False)

scoreNcust  scoreNsales  finalScore
Store1 Store2                                     
77.0   233.0     0.839021     0.712864    0.775943
       119.0     0.759688     0.597130    0.678409
       17.0      0.683098     0.629577    0.656337
       115.0     0.593774     0.717513    0.655643
       71.0      0.609718     0.691421    0.650569
...                   ...          ...         ...
       76.0           NaN          NaN         NaN
       85.0           NaN          NaN         NaN
       206.0          NaN          NaN         NaN
       211.0          NaN          NaN         NaN
       252.0          NaN          NaN         NaN

[271 rows x 3 columns]

Store 233 has highest score , so it is selected as control store for trial store 77

In [ ]:
# Now that we have found a control store, let's check visually if the drivers are indeed similar to store 77 before the trial period
# Set store 233 as 'control_store'

